<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-저평가기업-목록-읽기" data-toc-modified-id="1.-저평가기업-목록-읽기-1">1. 저평가기업 목록 읽기</a></span></li><li><span><a href="#2.-거래상위-종목-스크래핑" data-toc-modified-id="2.-거래상위-종목-스크래핑-2">2. 거래상위 종목 스크래핑</a></span></li><li><span><a href="#3.-저평가-&amp;-거래상위-종목-탐색" data-toc-modified-id="3.-저평가-&amp;-거래상위-종목-탐색-3">3. 저평가 &amp; 거래상위 종목 탐색</a></span></li></ul></div>

# 1. 저평가기업 목록 읽기

In [4]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv('./data/저평가기업목록.csv', index_col=0)
data.tail()

,회사명,종목코드,업종,업종2,업종PER,PER,PBR
362,대한제당,1790,"곡물가공품, 전분 및 전분제품 제조업",식품,29.07,9.23,0.40
363,삼양홀딩스,70,기타 금융업,식품,29.07,19.26,0.49
364,한국테크놀로지그룹,240,기타 금융업,자동차부품,46.67,18.53,0.61
365,메리츠화재,60,보험업,손해보험,6.87,4.41,0.61
366,경방,50,종합 소매업,"섬유,의류,신발,호화품",33.35,7.49,0.43


종목코드의 앞 자리 0이 모두 사라져 여섯 자리가 아닌 행들이 있으므로, map() 함수를 사용해 이를 보정

In [7]:
data.종목코드 = data.종목코드.map('{:06d}'.format)
data.tail()

,회사명,종목코드,업종,업종2,업종PER,PER,PBR
362,대한제당,001790,"곡물가공품, 전분 및 전분제품 제조업",식품,29.07,9.23,0.40
363,삼양홀딩스,000070,기타 금융업,식품,29.07,19.26,0.49
364,한국테크놀로지그룹,000240,기타 금융업,자동차부품,46.67,18.53,0.61
365,메리츠화재,000060,보험업,손해보험,6.87,4.41,0.61
366,경방,000050,종합 소매업,"섬유,의류,신발,호화품",33.35,7.49,0.43


# 2. 거래상위 종목 스크래핑

- **네이버**(https://finance.naver.com/sise/sise_quant.nhn?sosok=0) 사이트에서 **거래상위 테이블 수집**

당일거래량, 전일거래량(이틀치 거래량) 데이터 수집을 위해 동적인 Selenium을 활용할 예정

https://sites.google.com/a/chromium.org/chromedriver 에서 win32 버전 chromedriver 설치

In [8]:
from selenium import webdriver

In [12]:
# webdriver 경로
chromedriver = 'C:\\Users\\dalgo\\OneDrive\\바탕 화면\\08_Investment_Portfolio\\chromedriver.exe'

# selenium의 webdriver에 연동
driver = webdriver.Chrome(chromedriver)

driver.get('https://finance.naver.com/sise/sise_quant.nhn?sosok=0')

# 매수호가, 매도호가, ROE 버튼 해제 및 전일거래량 클릭
driver.find_element_by_xpath('//*[@id="option2"]').click()
driver.find_element_by_xpath('//*[@id="option8"]').click()
driver.find_element_by_xpath('//*[@id="option9"]').click()
driver.implicitly_wait(2)
driver.find_element_by_xpath('//*[@id="option12"]').click()
driver.find_element_by_xpath('//*[@id="option24"]').click()
driver.find_element_by_xpath('//*[@id="contentarea_left"]/div[2]/form/div/div/div/a[1]/img').click()

# 코스피 거래상위 수집
html = driver.page_source
table_0 = pd.read_html(html)[1]
table_0 = table_0.dropna(subset=['종목명']).drop(['N','Unnamed: 11'], axis=1).reset_index(drop=True)

# 코스닥 거래상위 수집
driver.implicitly_wait(2)
driver.find_element_by_xpath('//*[@id="contentarea"]/div[3]/div/div[2]/a').click()
html2 = driver.page_source
table_1 = pd.read_html(html2)[1]
table_1 = table_1.dropna(subset=['종목명']).drop(['N','Unnamed: 11'], axis=1).reset_index(drop=True)
table_1.head()

,종목명,현재가,전일비,등락률,거래량,거래대금,전일거래량,시가총액,PER,PBR
0,이화전기,320.0,0.0,0.00%,1.037267e+09,330446.0,484677142.0,2299.0,-24.62,1.23
1,이트론,834.0,99.0,+13.47%,9.036287e+08,699841.0,262437907.0,4375.0,-34.75,6.50
2,자안,354.0,59.0,-14.29%,2.911314e+08,112284.0,339519016.0,996.0,-5.21,1.38
3,세종텔레콤,716.0,0.0,0.00%,1.365789e+08,100791.0,382121828.0,4298.0,238.67,1.31
4,다날,7350.0,1330.0,-15.32%,9.896443e+07,759899.0,25749324.0,5068.0,45.94,2.13


In [14]:
table = pd.concat([table_0, table_1], axis=0).sort_values(by='거래량', ascending=False).reset_index(drop=True)
table.head()

,종목명,현재가,전일비,등락률,거래량,거래대금,전일거래량,시가총액,PER,PBR
0,서울식품,330.0,60.0,+22.22%,1.184037e+09,371437.0,373127685.0,1202.0,-82.50,4.54
1,이화전기,320.0,0.0,0.00%,1.037267e+09,330446.0,484677142.0,2299.0,-24.62,1.23
2,이아이디,502.0,45.0,-8.23%,9.143304e+08,472333.0,273589027.0,3631.0,-26.42,2.33
3,이트론,834.0,99.0,+13.47%,9.036287e+08,699841.0,262437907.0,4375.0,-34.75,6.50
4,KODEX 200선물인버스2X,2080.0,45.0,-2.12%,5.863714e+08,1255173.0,373295811.0,22256.0,NaN,NaN


In [15]:
table.to_csv('./data/네이버금융_거래량.csv', index=False)

# 3. 저평가 & 거래상위 종목 탐색

In [43]:
l = []
for i in table['종목명']:
    if i in data['회사명'].unique():
        l.append(i)
print(l)

['코디엠', '한화생명', 'KTcs', '골든센츄리', '경남스틸', 'NI스틸', '팬오션', '크리스탈신소재', 'KD', '글로벌에스엠', '로스웰', '문배철강', '티케이케미칼', 'S&K폴리텍', '팬스타엔터프라이즈', '대한해운', 'KTis', '삼표시멘트', '미래아이앤지', '패션플랫폼', 'LG유플러스', '코오롱', '대원화성', 'DB', '영풍제지']
